In [0]:
!apt-get --purge -y remove 'cuda*'
!apt-get --purge -y remove 'nvidia*'
!apt autoremove -y
!apt-get clean
!apt update -qq;
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.1.168-1_amd64.deb;
!dpkg -i cuda-repo-ubuntu1804_10.1.168-1_amd64.deb;
!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-10-1=10.1.168-1;

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'cuda-cusparse-cross-qnx-10-0' for glob 'cuda*'
Note, selecting 'cuda-npp-cross-qnx-10-0' for glob 'cuda*'
Note, selecting 'cuda-nvprune-10-0' for glob 'cuda*'
Note, selecting 'cuda-nvprune-10-1' for glob 'cuda*'
Note, selecting 'cuda-nvprune-10-2' for glob 'cuda*'
Note, selecting 'cuda-nvgraph-10-0' for glob 'cuda*'
Note, selecting 'cuda-nvgraph-10-1' for glob 'cuda*'
Note, selecting 'cuda-nvgraph-10-2' for glob 'cuda*'
Note, selecting 'cuda-cublas-dev-10-0' for glob 'cuda*'
Note, selecting 'cuda-gdb-10-0' for glob 'cuda*'
Note, selecting 'cuda-gdb-10-1' for glob 'cuda*'
Note, selecting 'cuda-gdb-10-2' for glob 'cuda*'
Note, selecting 'cuda-nsight-compute--10-0' for glob 'cuda*'
Note, selecting 'cuda-visual-tools-10-0' for glob 'cuda*'
Note, selecting 'cuda-visual-tools-10-1' for glob 'cuda*'
Note, selecting 'cuda-visual-tools-10-2' for glob 'cuda*'
Note, selecting 'cuda-cur

In [0]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-v1olgyx8
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-v1olgyx8
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=db8ce621ac5ebb365c3a80e4dfce39f246241a6f854a1a4058aa79a2330fff2d
  Stored in directory: /tmp/pip-ephem-wheel-cache-l6koroje/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [0]:
%%cu
#include <iostream>
int main() {
    std::cout << "Hello world\n";
    return 0;
}

Hello world



In [0]:
%%cu
#include <stdio.h>
#include <string>
#include <fstream>
#include <chrono>
#include <iostream>

const int width = 3840;
const int height = 2160;
const int max_iter = 1000;

class BMPWriter
{
public:
  enum ColorMode
  {
    RGB,
    BGR,
    BnW,
  };

public:
  struct FileHeader
  {
    unsigned char bfType[2] = { 'B', 'M' };
    unsigned char bfSize[4];
    unsigned char bfReserved1[2] = { 0, 0 };
    unsigned char bfReserved2[2] = { 0, 0 };
    unsigned char bfOffBits[4] = { 0, 0, 0, 0 };
  };

  struct InfoHeader
  {
    unsigned char biSize[4] = { 0, 0, 0, 0 };
    unsigned char biWidth[4];
    unsigned char biHeight[4];
    unsigned char biPlanes[2] = { 1, 0 };
    unsigned char biBitCount[2] = { 0, 0 };
    unsigned char biCompression[4] = { 0, 0, 0, 0 };
    unsigned char biSizeImage[4] = { 0, 0, 0, 0 };
    unsigned char biXPelsPerMeter[4] = { 0, 0, 0, 0 };
    unsigned char biYPelsPerMeter[4] = { 0, 0, 0, 0 };
    unsigned char biClrUsed[4] = { 0, 0, 0, 0 };
    unsigned char biClrImportant[4] = { 0, 0, 0, 0 };
  };

public:
  BMPWriter(size_t i_width, size_t i_height);
  void Write(const std::string& i_file_path,int *pic);

private:
  FILE* mp_file;
  size_t m_width;
  size_t m_height;
  size_t m_bytes_per_pixel;
  size_t m_file_size;
  ColorMode m_color_mode;
  FileHeader m_file_header;
  InfoHeader m_info_header;
};

BMPWriter::BMPWriter(size_t i_width, size_t i_height)
  : mp_file(nullptr)
  , m_width(i_width)
  , m_height(i_height)
  , m_bytes_per_pixel(3)
  , m_color_mode(RGB)
{
  size_t info_size = sizeof(FileHeader) + sizeof(InfoHeader);
  m_file_size = info_size + (m_bytes_per_pixel * i_width + (4 - (m_width*m_bytes_per_pixel) % 4) % 4) * i_height;
  for (auto i = 0u; i < 4; ++i)
  {
    m_file_header.bfSize[i] = static_cast<unsigned char>(m_file_size >> (i * 8));
    m_file_header.bfOffBits[i] = static_cast<unsigned char>(info_size >> (i * 8));
    m_info_header.biHeight[i] = static_cast<unsigned char>(m_height >> (i * 8));
    m_info_header.biWidth[i] = static_cast<unsigned char>(m_width >> (i * 8));
    m_info_header.biSize[i] = static_cast<unsigned char>(sizeof(InfoHeader) >> (i * 8));
  }
  m_info_header.biBitCount[0] = static_cast<unsigned char>(m_bytes_per_pixel * 8);
}

void BMPWriter::Write(const std::string& i_file_path, int* picture)
{
  unsigned char padding[3] = { 0, 0, 0 };
  int paddingSize = (4 - (m_width*m_bytes_per_pixel) % 4) % 4;
  mp_file = fopen(i_file_path.c_str(), "wb");
  fwrite(&m_file_header, sizeof(FileHeader), 1, mp_file);
  fwrite(&m_info_header, sizeof(InfoHeader), 1, mp_file);
  for (auto i = 0u; i < m_height; ++i)
    {
      for (auto j = 0u; j < m_width; ++j)
        for(int k =0;k<3;++k)
          fwrite(&picture[i*m_width*3+j*3+2-k], 1, 1, mp_file);
      fwrite(padding, 1, paddingSize, mp_file);
    }
  fclose(mp_file);
}

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

int get_iter(int x, int y)
{
    const float cx = 3.5 * x / width - 2.5;
    const float cy = 2.0 * y / height - 1.0;
    float zx = 0;
    float zy = 0;
    int iter = 0;
    while (iter < max_iter)
      {
        const float tempzx = zx * zx - zy * zy + cx;
        zy = 2 * zx * zy + cy;
        zx = tempzx;
        if(zx * zx + zy * zy > 4)
          break;
        ++iter;
      }
    return iter;
}

__global__ void mandelbrot_set(int* i_color_map, int* o_picture)
	{
	const int x = threadIdx.x + blockIdx.x * blockDim.x;
	const int y = threadIdx.y + blockIdx.y * blockDim.y;
  if(x>=width || y>=height)
    return;
	const int pixel_coords = y * width*3 + x*3;
  const float cx = 3.5 * x / width - 2.5;
	const float cy = 2.0 * y / height - 1.0;
	float zx = 0;
	float zy = 0;
	int iter = 0;
	while (iter < max_iter)
		{
		const float tempzx = zx * zx - zy * zy + cx;
		zy = 2 * zx * zy + cy;
		zx = tempzx;
    if(zx * zx + zy * zy > 4)
      break;
		++iter;
		}
  int id = 0;
  if(iter != max_iter)
    id = (iter * 100 / max_iter) % 17;
	o_picture[pixel_coords + 0] = i_color_map[id * 3 + 0];
	o_picture[pixel_coords + 1] = i_color_map[id * 3 + 1];
	o_picture[pixel_coords + 2] = i_color_map[id * 3 + 2];
	}

void mandelbrot_set_cpu(int* i_color_map, int* o_picture)
	{
  for(int y = 0;y<height;++y)
    for(int x=0;x<width;++x)
	    {
        const int pixel_coords = y * width*3 + x*3;
	      int iter = get_iter(x,y);
        int id = 0;
        if(iter != max_iter)
          id = (iter * 100 / max_iter) % 17;
        o_picture[pixel_coords + 0] = i_color_map[id * 3 + 0];
	      o_picture[pixel_coords + 1] = i_color_map[id * 3 + 1];
	      o_picture[pixel_coords + 2] = i_color_map[id * 3 + 2];
      }
	}

int main()
{
  int color_map[17 * 3] =
    {
    0, 0, 0,
    66, 45, 15,
    25, 7, 25,
    10, 0, 45,
    5, 5, 73,
    0, 7, 99,
    12, 43, 137,
    22, 81, 175,
    56, 124, 209,
    132, 181, 229,
    209, 234, 247,
    239, 232, 191,
    247, 201, 94,
    255, 170, 0,
    204, 127, 0,
    153, 86, 0,
    104, 51, 2,
    };
 auto t1 = std::chrono::system_clock::now();
  int *cmap,*c_cpu, *c_gpu;
  int size = 3*width*height* sizeof (int);
  int sz = 51*sizeof(int);
  cudaMallocManaged (&cmap, sz);
  cudaMallocManaged (&c_cpu, size);
  cudaMallocManaged (&c_gpu, size);
 
 for(int i=0;i<51;++i)
  cmap[i] = color_map[i];

  for( int y = 0; y < height; ++y )
    for( int x = 0; x < width; ++x )
      for(int k =0;k<3;++k)
        {
          c_cpu[y*width*3 + x*3+k] = 0;
          c_gpu[y*width*3 + x*3+k] = 0;
        }
 
  dim3 threads_per_block(32, 32, 1);
  dim3 number_of_blocks(width / threads_per_block.x + 1, height / threads_per_block.y + 1, 1);
  mandelbrot_set <<< number_of_blocks, threads_per_block >>> ( cmap, c_gpu );
 auto t2 = std::chrono::system_clock::now();
 std::cout<<"GPU time: "<<std::chrono::duration_cast<std::chrono::milliseconds>(t2-t1).count()<<std::endl;
  checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
  checkCudaErr(cudaGetLastError(), "GPU");
 t1 = std::chrono::system_clock::now();
  mandelbrot_set_cpu( cmap, c_cpu );
 t2 = std::chrono::system_clock::now();
 std::cout<<"CPU time: "<<std::chrono::duration_cast<std::chrono::milliseconds>(t2-t1).count()<<std::endl;
 
  bool error = false;
  for( int y = 0; y < height && !error; ++y )
    for( int x = 0; x < width && !error; ++x )
      if (c_cpu[y * width + x] != c_gpu[y * width + x])
      {
        printf("FOUND ERROR at c[%d][%d]\n", y, x);
        error = true;
        break;
      }
 BMPWriter writer(width,height);
 writer.Write("test_cpu.bmp",c_cpu);
 writer.Write("test_gpu.bmp",c_gpu);
  if (!error)
    printf("Success!\n");
  cudaFree(cmap);
  cudaFree( c_cpu );
  cudaFree( c_gpu );
}

GPU time: 375
CPU time: 14411
FOUND ERROR at c[53][408]



In [0]:
from google.colab import files
uploaded = files.upload()

MessageError: ignored

In [0]:
!ls

a.out  cuda  cuda.cu  sample_data
